<a href="https://colab.research.google.com/github/hyeok1235/PyTorch/blob/main/Build_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Neural networks comprise of layers/modules that perform operations on data.The torch.nn namespace provides all the building blocks you need to build your own neural network.

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Get Device for Training

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


# Define the Class

In [3]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10),
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

In [5]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [7]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([2], device='cuda:0')


# Model Layers

In [8]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


In [9]:
# nn.Flatten
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [10]:
# nn.Linear
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [11]:
# nn.ReLU
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.2581, -0.0684,  0.2923, -0.2242, -0.2664,  0.3444,  0.1031, -0.5133,
         -0.4467,  0.0754,  0.2917, -0.8381,  0.0517,  0.0483, -0.1029, -0.1462,
          0.2315, -0.6482,  0.0801,  0.3193],
        [ 0.1721,  0.2776,  0.3704, -0.1828, -0.0133,  0.8159,  0.0769,  0.1036,
         -0.0902, -0.0754, -0.0912, -0.2719, -0.1928,  0.2100, -0.1265,  0.0268,
          0.0988, -0.5558,  0.3948,  0.0060],
        [-0.0557, -0.0706,  0.4288,  0.0362, -0.2516,  0.4984,  0.1592, -0.3293,
         -0.0572, -0.1216, -0.0531, -0.3031, -0.1623,  0.1662,  0.0084,  0.2665,
          0.1630, -0.3069,  0.3317,  0.1481]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.2581, 0.0000, 0.2923, 0.0000, 0.0000, 0.3444, 0.1031, 0.0000, 0.0000,
         0.0754, 0.2917, 0.0000, 0.0517, 0.0483, 0.0000, 0.0000, 0.2315, 0.0000,
         0.0801, 0.3193],
        [0.1721, 0.2776, 0.3704, 0.0000, 0.0000, 0.8159, 0.0769, 0.1036, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.2100, 0.00

In [12]:
# nn.Sequential
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

In [15]:
# nn.Softmax
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

# Model Parameters

In [17]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
  print(f"Layer: {name} | Size: {param.size()} | Values: {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[ 0.0341, -0.0185, -0.0103,  ..., -0.0034,  0.0167, -0.0245],
        [ 0.0094,  0.0114,  0.0253,  ...,  0.0099,  0.0314,  0.0183]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values: tensor([0.0143, 0.0297], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values: tensor([[-0.0013,  0.0420, -0.0430,  ..., -0.0380,  0.0010, -0.0248],
        [-0.0005,  0.0246, -0.0437,  ..., -0.0302,  0.0424, -0.0116]],
       device='cuda:0', grad_fn=<SliceBa